In [1]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from IPython.display import clear_output

from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import Estimator
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.connectors import TorchConnector


In [5]:
# ----- 构建 1D 横场 Ising Hamiltonian -----
def build_ising_hamiltonian(n, J, h):
    paulis = []
    coeffs = []

    # Z_i Z_{i+1} 项
    for i in range(n - 1):
        label = ['I'] * n
        label[i] = 'Z'
        label[i + 1] = 'Z'
        paulis.append(''.join(reversed(label)))  # Qiskit从右往左
        coeffs.append(-J)

    # X_i 项
    for i in range(n):
        label = ['I'] * n
        label[i] = 'X'
        paulis.append(''.join(reversed(label)))
        coeffs.append(-h)

    return SparsePauliOp.from_list(list(zip(paulis, coeffs)))

In [7]:
def create_physically_inspired_ansatz(n_qubits, h, J, reps=3):
    params = ParameterVector('θ', length=2 * n_qubits * reps)
    qc = QuantumCircuit(n_qubits)

    # 初态准备：根据 h/J 设置 RY 角度，模拟磁化方向
    init_angle = 2 * np.arctan(h / J)  # 乘 2 是因为 RY(θ)|0> = cos(θ/2)|0> + sin(θ/2)|1>
    for i in range(n_qubits):
        qc.ry(init_angle, i)

    # Ansatz 部分：多层旋转+纠缠结构
    for rep in range(reps):
        for i in range(n_qubits):
            qc.ry(params[2 * n_qubits * rep + i], i)
            qc.rx(params[2 * n_qubits * rep + n_qubits + i], i)
        for i in range(n_qubits - 1):
            qc.cx(i, i + 1)

    return qc, list(params)

In [10]:
# 使用 Adagrad 优化器
'''optimizer = torch.optim.Adagrad(qnn_model.parameters(), lr=0.1)  # 可调整学习率'''

'optimizer = torch.optim.Adagrad(qnn_model.parameters(), lr=0.1)  # 可调整学习率'

In [11]:
# 使用随机梯度下降 (SGD) 优化器
'''optimizer = torch.optim.SGD(qnn_model.parameters(), lr=0.01)  # 可根据需要调整学习率'''


'optimizer = torch.optim.SGD(qnn_model.parameters(), lr=0.01)  # 可根据需要调整学习率'

In [17]:
# 对N=2到10分别计算基态能量和磁化率
results = []
for n_qubits in range(2, 11):
    J = 1.0
    h = 1.0
    hamiltonian = build_ising_hamiltonian(n_qubits, J, h)
    ansatz_circuit, ansatz_params = create_physically_inspired_ansatz(n_qubits, h, J, reps=4)
    estimator = Estimator()
    qnn = EstimatorQNN(
        circuit=ansatz_circuit,
        observables=hamiltonian,
        input_params=[],
        weight_params=ansatz_params,
        estimator=estimator
    )
    qnn_model = TorchConnector(qnn)
    optimizer = torch.optim.AdamW(qnn_model.parameters(), lr=0.01)
   
    # 训练模型
    for epoch in range(500):  # 迭代步数可调
        optimizer.zero_grad()
        output = qnn_model()
        loss = output.mean()  # 只优化能量
        loss.backward()
        optimizer.step()

    # 记录最优参数下的物理量
    final_weights = qnn_model.weight.detach().numpy()
    estimator = Estimator()
    # <H> 基态能量
    E0 = estimator.run(circuits=ansatz_circuit, observables=hamiltonian, parameter_values=[final_weights]).result().values[0]

    # <Z> 总磁化
    z_obs = SparsePauliOp.from_list([(f"{'I'*i + 'Z' + 'I'*(n_qubits - i - 1)}", 1.0) for i in range(n_qubits)])
    mz = estimator.run(circuits=ansatz_circuit, observables=z_obs, parameter_values=[final_weights]).result().values[0] / n_qubits

    # <X> 总磁化
    x_obs = SparsePauliOp.from_list([(f"{'I'*i + 'X' + 'I'*(n_qubits - i - 1)}", 1.0) for i in range(n_qubits)])
    mx = estimator.run(circuits=ansatz_circuit, observables=x_obs, parameter_values=[final_weights]).result().values[0] / n_qubits
    results.append((n_qubits, E0, mz, mx))
    
    print(f"N={n_qubits}, E0={E0:.6f}, <Z>={mz:.6f}, <X>={mx:.6f}")



/var/folders/d9/_3jh3gcd7m5gl50nsj1mgls80000gn/T/ipykernel_5982/267513214.py:9: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = EstimatorQNN(


N=2, E0=-2.236068, <Z>=0.000281, <X>=0.894537
N=3, E0=-3.493832, <Z>=0.005369, <X>=0.842585
N=3, E0=-3.493832, <Z>=0.005369, <X>=0.842585
N=4, E0=-4.756895, <Z>=0.035552, <X>=0.811119
N=4, E0=-4.756895, <Z>=0.035552, <X>=0.811119
N=5, E0=-6.025982, <Z>=-0.000959, <X>=0.788909
N=5, E0=-6.025982, <Z>=-0.000959, <X>=0.788909
N=6, E0=-7.292277, <Z>=0.000726, <X>=0.773863
N=6, E0=-7.292277, <Z>=0.000726, <X>=0.773863
N=7, E0=-8.560426, <Z>=-0.000095, <X>=0.761768
N=7, E0=-8.560426, <Z>=-0.000095, <X>=0.761768
N=8, E0=-9.825080, <Z>=-0.000248, <X>=0.754983
N=8, E0=-9.825080, <Z>=-0.000248, <X>=0.754983
N=9, E0=-11.088728, <Z>=-0.000720, <X>=0.749347
N=9, E0=-11.088728, <Z>=-0.000720, <X>=0.749347
N=10, E0=-12.355604, <Z>=0.004181, <X>=0.744313
N=10, E0=-12.355604, <Z>=0.004181, <X>=0.744313


In [18]:
# 汇总输出
print("\nSummary for N=2 to 10:")
for n, E0, mz, mx in results:
    print(f"N={n}: E0={E0:.6f}, <Z>={mz:.6f}, <X>={mx:.6f}")


Summary for N=2 to 10:
N=2: E0=-2.236068, <Z>=0.000281, <X>=0.894537
N=3: E0=-3.493832, <Z>=0.005369, <X>=0.842585
N=4: E0=-4.756895, <Z>=0.035552, <X>=0.811119
N=5: E0=-6.025982, <Z>=-0.000959, <X>=0.788909
N=6: E0=-7.292277, <Z>=0.000726, <X>=0.773863
N=7: E0=-8.560426, <Z>=-0.000095, <X>=0.761768
N=8: E0=-9.825080, <Z>=-0.000248, <X>=0.754983
N=9: E0=-11.088728, <Z>=-0.000720, <X>=0.749347
N=10: E0=-12.355604, <Z>=0.004181, <X>=0.744313
